<a href="https://colab.research.google.com/github/vrihatgan/SillyTavern/blob/release/colab/sillywithkobold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#There exist two colob notebooks
#1.  Koboldcpp
https://colab.research.google.com/github/vrihatgan/SillyTavern/blob/release/colab/sillywithkobold.ipynb
#2.  Gemini api
first get your gemini key from here :--> https://aistudio.google.com/app/u/2/apikey
https://colab.research.google.com/github/vrihatgan/SillyTavern/blob/gemini/colab/sillywithgemini.ipynb

In [ ]:
#@title Tap this if you play on Mobile { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start KoboldCpp below</b><br/>
<audio autoplay="" src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" loop controls>

In [ ]:
#@title Tap this to run sillytavern { check box below if you want to back up on google drive } { display-mode: "form" }
Model = "https://huggingface.co/TheBloke/Noromaid-13B-v0.3-GGUF/resolve/main/noromaid-13b-v0.3.Q4_K_M.gguf"
use_drive_for_chat_backup = False #@param {type:"boolean"}
import os
import subprocess
import json
import time
import shutil
import threading
import sys
import torch
import requests
if not torch.cuda.is_available():
    from IPython.display import display, Markdown
    display(Markdown("<h1>NO free gpu is avilable for you at the moment</h1>"))
    sys.exit()
from google.colab import drive
os.chdir("/content")
print("SillyTavern brought to you by Cohee1207")
print("Colab notebook written by doctord98 with added api support by vrihatgan")
print("Please wait while I setup everything for you")
if use_drive_for_chat_backup:
  drive.mount('/content/drive')
subprocess.call("git clone https://github.com/vrihatgan/SillyTavern", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
src1 = '/content/SillyTavern/data/default-user/characters'
des1 = '/content/drive/MyDrive/SillyTavern/characters'
src2 = '/content/SillyTavern/data/default-user/chats'
des2 = '/content/drive/MyDrive/SillyTavern/chats'
Layers = 99
ContextSize = 4096
subprocess.run("wget -O dlfile.tmp https://kcpplinux.concedo.workers.dev && mv dlfile.tmp koboldcpp_linux", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run("chmod +x ./koboldcpp_linux", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run("apt install aria2 -y", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print("downloading model")
subprocess.run(f"aria2c -x 10 -o model.gguf --summary-interval=5 --download-result=default --allow-overwrite=true --file-allocation=none {Model}", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print("downloading complete")
print("now loading model")
command = f"./koboldcpp_linux model.gguf --usecublas 0 mmq --multiuser --gpulayers {Layers} --contextsize {ContextSize} --quiet"
subprocess.Popen(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

flag_file = "backup_flag.txt"
if use_drive_for_chat_backup:
    if os.path.exists(flag_file):
        print("The code has already been executed.")
    else:

      if not os.path.exists(des1):
        os.makedirs(des1)
        shutil.copytree(src1, des1, dirs_exist_ok=True)
      else:
        !rm -rf /content/SillyTavern/data/default-user/characters/*
        shutil.copytree(des1, src1, dirs_exist_ok=True)
      if not os.path.exists(des2):
        os.makedirs(des2)
        shutil.copytree(src2, des2, dirs_exist_ok=True)
      else:
        !rm -rf /content/SillyTavern/data/default-user/chats/*
        shutil.copytree(des2, src2, dirs_exist_ok=True)



      def backup_thread():
        while True:
          try:
            shutil.copytree(src1, des1, dirs_exist_ok=True)
            shutil.copytree(src2, des2, dirs_exist_ok=True)
          except Exception as e:
            pass

          time.sleep(5)

      backup_thread = threading.Thread(target=backup_thread)
      backup_thread.start()

      with open(flag_file, "w") as f:
        f.write("you don't need to run this code again just run the code below if link not working")
subprocess.call("wget https://nodejs.org/dist/v20.15.1/node-v20.15.1-linux-x64.tar.xz", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.call("tar -xvf node-v20.15.1-linux-x64.tar.xz", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.call("mv node-v20.15.1-linux-x64 node", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.call("sudo cp -R node/* /usr/local/", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
!rm -rf node-v20.15.1-linux-x64.tar.xz node
os.chdir("/content/SillyTavern")
subprocess.call("npm install", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
os.chdir('/content/')
if not os.path.isfile('cloudflared-linux-amd64'):
    os.system('wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64')
    os.chmod('cloudflared-linux-amd64', 0o777)

os.system('nohup ./cloudflared-linux-amd64 tunnel --url http://127.0.0.1:8000 --metrics 127.0.0.1:31337 > cf.txt 2>&1 &')
time.sleep(4)

scrape = requests.get('http://127.0.0.1:31337/metrics').text
needle = scrape.partition('cloudflared_tunnel_user_hostnames_counts{userHostname="')[2].split('"} 1')[0]

if needle:
    with open('cloudflare.log', 'w') as log:
        log.write("CLOUDFLARE PROVIDES!" + needle)
print(needle)
os.chdir("/content/SillyTavern")
!nohup node server.js >/dev/null 2>&1